In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

distributed 1.21.8 requires msgpack, which is not installed.
Libraries imported.


In [3]:
# To extract from Wikipedia page
from bs4 import BeautifulSoup
url=  requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#with open (url) as html_file:
data = BeautifulSoup(url.text,'lxml')
#print(data.prettify())


In [4]:
#parse data and save to csv file
import csv
csv_file=open('canada.csv','w')
csv_writer=csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])
for tr in data.find_all('tr')[1:]:
    tds = tr.find_all('td')
    if len(tds)==3:
        Postcode=tds[0].text
        Borough=tds[1].text
        Neighbourhood=tds[2].text
    #print(Postcode, Borough, Neighbourhood)
    csv_writer.writerow([Postcode, Borough, Neighbourhood])
csv_file.close()

In [5]:
#load canada data from CSV
import pandas as pd
df_Canada=pd.read_csv('canada.csv')
print('Data loaded')

Data loaded


In [6]:
#removes \n at the end
df_Canada[df_Canada.columns] = df_Canada.apply(lambda x: x.str.strip('\n'))
df_Canada.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
# exclude not assigned borough 
df_Canada_Assigned=df_Canada[df_Canada.Borough!='Not assigned']
df_Canada_Assigned.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [8]:
#Group neighbourhoods by postal code
df_Canada_Assigned.set_index(['Postcode','Borough'],inplace=True)
result = df_Canada_Assigned.groupby(level=['Postcode','Borough'], sort=False).agg( ','.join)
result.reset_index() # to remove set index

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [9]:
#remove not assignet neighbourhood
result.loc[result['Neighbourhood'] =='Not assigned']=result[result['Neighbourhood'] == 'Not assigned'].index.values[0][1]
result.head()

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park


In [10]:
#make a dataframe for canada
df_Canada_final=result.reset_index()
df_Canada_final.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
df_Canada_final.shape 

(103, 3)

In [12]:
#format address to be searchable in google maps
df_Canada_final["Address"]=df_Canada_final['Borough'] +", " +df_Canada_final['Postcode'] 
df_Canada_final.head()

,Postcode,Borough,Neighbourhood,Address
0,M3A,North York,Parkwoods,"North York, M3A"
1,M4A,North York,Victoria Village,"North York, M4A"
2,M5A,Downtown Toronto,"Harbourfront,Regent Park","Downtown Toronto, M5A"
3,M6A,North York,"Lawrence Heights,Lawrence Manor","North York, M6A"
4,M7A,Queen's Park,Queen's Park,"Queen's Park, M7A"


ASIGNMENT PART 2

In [13]:
#READ GEO DATA
geoData=pd.read_csv("https://cocl.us/Geospatial_data")
geoData

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [15]:
# CONCAT DATA AND GEODATA
geoReferenced = pd.concat([df_Canada_final, geoData], axis=1)
geoReferenced.drop('Postal Code', axis=1, inplace=True)
geoReferenced

,Postcode,Borough,Neighbourhood,Address,Latitude,Longitude
0,M3A,North York,Parkwoods,"North York, M3A",43.806686,-79.194353
1,M4A,North York,Victoria Village,"North York, M4A",43.784535,-79.160497
2,M5A,Downtown Toronto,"Harbourfront,Regent Park","Downtown Toronto, M5A",43.763573,-79.188711
3,M6A,North York,"Lawrence Heights,Lawrence Manor","North York, M6A",43.770992,-79.216917
4,M7A,Queen's Park,Queen's Park,"Queen's Park, M7A",43.773136,-79.239476
5,M9A,Etobicoke,Islington Avenue,"Etobicoke, M9A",43.744734,-79.239476
6,M1B,Scarborough,"Rouge,Malvern","Scarborough, M1B",43.727929,-79.262029
7,M3B,North York,Don Mills North,"North York, M3B",43.711112,-79.284577
8,M4B,East York,"Woodbine Gardens,Parkview Hill","East York, M4B",43.716316,-79.239476
9,M5B,Downtown Toronto,"Ryerson,Garden District","Downtown Toronto, M5B",43.692657,-79.264848


In [17]:
geoReferenced.shape

(103, 6)

ASIGNMENT PART 3

In [19]:
# TORONTO EXPLORATION
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(geoReferenced['Borough'].unique()),
        geoReferenced.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [20]:
borough = geoReferenced['Borough'].unique()
borough

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [22]:
address = 'Toronto, ON, Canada'
geolocator = Nominatim (user_agent="my-application")
location = geolocator.geocode (address)
latitude = location.latitude 
longitude = location.longitude
print ('The geographical coordinate of Toronto, Canada are: {},{} '.format(latitude,longitude))

The geographical coordinate of Toronto, Canada are: 43.653963,-79.387207 


In [26]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, postCode, borough, neighborhood in zip(geoReferenced['Latitude'], 
                                                     geoReferenced['Longitude'],
                                                     geoReferenced['Postcode'],
                                                     geoReferenced['Borough'], 
                                                     geoReferenced['Neighbourhood']):
    label = '{} in {} {}'.format(neighborhood, borough,postCode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [31]:
etobicoke=geoReferenced[geoReferenced['Borough'].str.contains("Etobicoke")].reset_index(drop=True)
etobicoke.drop('Postcode', axis=1, inplace=True)
etobicoke.head(10)

,Borough,Neighbourhood,Address,Latitude,Longitude
0,Etobicoke,Islington Avenue,"Etobicoke, M9A",43.744734,-79.239476
1,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...","Etobicoke, M9B",43.750072,-79.295849
2,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...","Etobicoke, M9C",43.803762,-79.363452
3,Etobicoke,Westmount,"Etobicoke, M9P",43.648429,-79.382280
4,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...","Etobicoke, M9R",43.647927,-79.419750
5,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto","Etobicoke, M8V",43.605647,-79.501321
6,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...","Etobicoke, M9V",43.602414,-79.543484
7,Etobicoke,"Alderwood,Long Branch","Etobicoke, M8W",43.667856,-79.532242
8,Etobicoke,Northwest,"Etobicoke, M9W",43.650943,-79.554724
9,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North","Etobicoke, M8X",43.706876,-79.518188


In [33]:
address = 'Etobicoke, CA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


In [36]:
map_Etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, borough, neighborhood in zip(etobicoke['Latitude'], 
                                                     etobicoke['Longitude'],
                                                     etobicoke['Borough'], 
                                                     etobicoke['Neighbourhood']):
    label = '{} in {} {}'.format(neighborhood, borough,postCode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Etobicoke)  
    
map_Etobicoke

In [37]:
#Define Foursquare Credentials and Version
# 
CLIENT_ID = 'DNKGBW0DNDMPBVNWGOWNFJBBNB2JCRA1E55P5HH0GJL15BDT' # your Foursquare ID
CLIENT_SECRET = 'VDK2T3LF4QDMH2ISZOESF0MZPVR0U2ROFPJZMUGP4KHLJO4K' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: DNKGBW0DNDMPBVNWGOWNFJBBNB2JCRA1E55P5HH0GJL15BDT
CLIENT_SECRET:VDK2T3LF4QDMH2ISZOESF0MZPVR0U2ROFPJZMUGP4KHLJO4K


In [38]:
#Let's explore the first neighborhood in our dataframe.


etobicoke.loc[0, 'Neighbourhood']

'Islington Avenue'

In [39]:
#Get the neighborhood's latitude and longitude values.

neighborhood_latitude = etobicoke.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = etobicoke.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = etobicoke.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Islington Avenue are 43.7447342, -79.23947609999999.


In [40]:
#Now, let's get the top 50 venues that are in Islington Avenue within a radius of 500 meters.

LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius


 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=DNKGBW0DNDMPBVNWGOWNFJBBNB2JCRA1E55P5HH0GJL15BDT&client_secret=VDK2T3LF4QDMH2ISZOESF0MZPVR0U2ROFPJZMUGP4KHLJO4K&v=20180604&ll=43.7447342,-79.23947609999999&radius=500&limit=50'

In [41]:
resultsFSQ = requests.get(url).json()
resultsFSQ

{'meta': {'code': 200, 'requestId': '5bd92b29dd5797146b350b02'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Eglinton East',
  'headerFullLocation': 'Eglinton East, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.749234204500006,
    'lng': -79.23325872538938},
   'sw': {'lat': 43.7402341955, 'lng': -79.2456934746106}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5150a8dae4b045dfb6581c85',
       'name': 'McCowan Park',
       'location': {'lat': 43.74508851212816,
        'lng': -79.239335687338,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.7450885

In [42]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

In [43]:
venues = resultsFSQ['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,McCowan Park,Playground,43.745089,-79.239336
1,Tumbe Cafe,Grocery Store,43.744058,-79.244021


In [44]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [78]:
#Now, let's explore Neighborhoods in Etobicoke:

#Let's create a function to repeat the same process to all the neighborhoods in Etobicoke


def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [79]:
etobicoke_venues = getNearbyVenues(names=etobicoke['Neighbourhood'],
                                   latitudes=etobicoke['Latitude'],
                                   longitudes=etobicoke['Longitude']
                                  )

Islington Avenue
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Westmount
Kingsview Village,Martin Grove Gardens,Richview Gardens,St. Phillips
Humber Bay Shores,Mimico South,New Toronto
Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown
Alderwood,Long Branch
Northwest
The Kingsway,Montgomery Road,Old Mill North
Humber Bay,King's Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea
Kingsway Park South West,Mimico NW,The Queensway West,Royal York South West,South of Bloor


In [80]:
print(etobicoke_venues.shape)
etobicoke_venues.head()

(222, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Islington Avenue,43.744734,-79.239476,Diamond Pizza,43.743699,-79.245922,Pizza Place
1,Islington Avenue,43.744734,-79.239476,Dairy Queen Restaurant,43.739506,-79.236894,Ice Cream Shop
2,Islington Avenue,43.744734,-79.239476,DQ Grill & Chill Restaurant,43.739580,-79.236991,Ice Cream Shop
3,Islington Avenue,43.744734,-79.239476,Tim Hortons,43.738992,-79.238912,Coffee Shop
4,Islington Avenue,43.744734,-79.239476,Subway,43.738451,-79.236400,Sandwich Place


In [81]:
#Let's check how many venues were returned for each neighborhood

etobicoke_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",13,13,13,13,13,13
"Alderwood,Long Branch",9,9,9,9,9,9
"Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe",18,18,18,18,18,18
"Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park",20,20,20,20,20,20
"Humber Bay Shores,Mimico South,New Toronto",20,20,20,20,20,20
"Humber Bay,King's Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea",13,13,13,13,13,13
Islington Avenue,9,9,9,9,9,9
"Kingsview Village,Martin Grove Gardens,Richview Gardens,St. Phillips",50,50,50,50,50,50
"Kingsway Park South West,Mimico NW,The Queensway West,Royal York South West,South of Bloor",1,1,1,1,1,1


In [82]:
#Let's find out how many unique categories 


print('There are {} uniques categories.'.format(len(etobicoke_venues['Venue Category'].unique())))

There are 87 uniques categories.


In [83]:
#Now, let's analyze Each Neighborhood

# one hot encoding
etobicoke_onehot = pd.get_dummies(etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etobicoke_onehot['Neighbourhood'] = etobicoke_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [etobicoke_onehot.columns[-1]] + list(etobicoke_onehot.columns[:-1])
etobicoke_onehot = etobicoke_onehot[fixed_columns]

etobicoke_onehot.head()

,Neighbourhood,African Restaurant,American Restaurant,Art Gallery,Asian Restaurant,Auto Garage,BBQ Joint,Bakery,Bank,Bar,Beer Store,Bistro,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food Court,French Restaurant,Fried Chicken Joint,Gas Station,Gastropub,General Travel,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hookah Bar,Hotel,Housing Development,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Liquor Store,Lounge,Malay Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Museum,Music Venue,New American Restaurant,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pub,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Speakeasy,Steakhouse,Theater,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Islington Avenue,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Islington Avenue,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Islington Avenue,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Islington Avenue,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Islington Avenue,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [84]:
etobicoke_onehot.shape

(222, 88)

In [85]:
etobicoke_grouped = etobicoke_onehot.groupby('Neighbourhood').mean().reset_index()
etobicoke_grouped

,Neighbourhood,African Restaurant,American Restaurant,Art Gallery,Asian Restaurant,Auto Garage,BBQ Joint,Bakery,Bank,Bar,Beer Store,Bistro,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food Court,French Restaurant,Fried Chicken Joint,Gas Station,Gastropub,General Travel,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hookah Bar,Hotel,Housing Development,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Liquor Store,Lounge,Malay Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Museum,Music Venue,New American Restaurant,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pub,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Speakeasy,Steakhouse,Theater,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.076923,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.076923,0.00,0.076923,0.00,0.00,0.00,0.00,0.000000,0.076923,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.076923,0.00,0.00,0.00,0.000000,0.076923,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.076923,0.00,0.076923,0.153846,0.000000,0.00,0.076923,0.00,0.000000,0.000000,0.000000,0.00,0.076923,0.00,0.000000,0.076923,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00
1,"Alderwood,Long Branch",0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.111111,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.111111,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.111111,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.111111,0.00,0.222222,0.000000,0.111111,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.111111,0.111111,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00
2,"Bloordale Gardens,Eringate,Markland Wood,Old B...",0.00,0.000000,0.00,0.00,0.000000,0.00,0.055556,0.055556,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.055556,0.000000,0.00,0.055556,0.00,0.000000,0.00,0.00,0.00,0.00,0.055556,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.055556,0.055556,0.00,0.00,0.00,0.055556,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.111111,0.00,0.166667,0.055556,0.000000,0.00,0.000000,0.00,0.055556,0.055556,0.000000,0.00,0.055556,0.00,0.055556,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00
3,"Cloverdale,Islington,Martin Grove,Princess Gar...",0.05,0.000000,0.00,0.00,0.000000,0.00,0.050000,0.000000,0.05,0.000000,0.00,0.05,0.00,0.05,0.000000,0.00,0.050000,0.000000,0.000000,0.00,0.050000,0.00,0.050000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.05,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.050000,0.000000,0.00,0.000000,0.05,0.000000,0.000000,0.000000,0.05,0.00,0.00,0.050000,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.050000,0.100000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.050000,0.00,0.000000,0.05,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.05,0.00,0.00
4,"Humber Bay Shores,Mimico South,New Toronto",0.00,0.050000,0.00,0.00,0.000000,0.00,0.050000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.05,0.050000,0.000000,0.000000,0.00,0.000000,0.00,0.050000,0.00,0.00,0.00,0.05,0.000000,0.000000,0.05,0.050000,0.05,0.00,0.00,0.00,0.050000,0.00

In [86]:
etobicoke_grouped.shape

(12, 88)

In [87]:
#Let's print each neighborhood along with the top 5 most common venues of Etobicoke

num_top_venues = 5

for hood in etobicoke_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = etobicoke_grouped[etobicoke_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
               venue  freq
0        Pizza Place  0.15
1     Sandwich Place  0.08
2       Skating Rink  0.08
3           Pharmacy  0.08
4  Convenience Store  0.08


----Alderwood,Long Branch----
           venue  freq
0       Pharmacy  0.22
1           Bank  0.11
2           Park  0.11
3           Café  0.11
4  Shopping Mall  0.11


----Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe----
            venue  freq
0        Pharmacy  0.17
1            Park  0.11
2  Ice Cream Shop  0.06
3   Shopping Mall  0.06
4           Diner  0.06


----Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park----
                venue  freq
0         Pizza Place  0.10
1  African Restaurant  0.05
2  Seafood Restaurant  0.05
3   Korean Restaurant  0.05
4          Hookah Bar  0.05


----Humber Bay Shores,Mimico South,New Toronto----
                venue  freq
0            Bus S

In [88]:
#Sort the venues in descending order.


def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [89]:
#Display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = etobicoke_grouped['Neighbourhood']

for ind in np.arange(etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Pizza Place,Park,Pub,Gym,Convenience Store,Gas Station,Pharmacy,Skating Rink,Bank,Sandwich Place
1,"Alderwood,Long Branch",Pharmacy,Park,Grocery Store,Café,Playground,Skating Rink,Shopping Mall,Bank,Donut Shop,Cuban Restaurant
2,"Bloordale Gardens,Eringate,Markland Wood,Old B...",Pharmacy,Park,Residential Building (Apartment / Condo),Sandwich Place,Chinese Restaurant,Pizza Place,Korean Restaurant,Diner,Recreation Center,Ice Cream Shop
3,"Cloverdale,Islington,Martin Grove,Princess Gar...",Pizza Place,African Restaurant,Bar,Convenience Store,Coffee Shop,Korean Restaurant,Café,Pharmacy,Burger Joint,Intersection
4,"Humber Bay Shores,Mimico South,New Toronto",Mexican Restaurant,Falafel Restaurant,Pharmacy,Pizza Place,Café,Bus Stop,Dessert Shop,Liquor Store,Italian Restaurant,Convenience Store
5,"Humber Bay,King's Mill Park,Kingsway Park Sout...",Grocery Store,Pizza Place,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Coffee Shop,Pharmacy,Sandwich Place,Hardware Store,Bar
6,Islington Avenue,Ice Cream Shop,Coffee Shop,Convenience Store,Auto Garage,Pizza Place,Sandwich Place,Restaurant,Fast Food Restaurant,Falafel Restaurant,Cuban Restaurant
7,"Kingsview Village,Martin Grove Gardens,Richvie...",Bar,Café,Coffee Shop,Bakery,Cocktail Bar,Pizza Place,Restaurant,French Restaurant,Asian Restaurant,Vietnamese Restaurant
8,"Kingsway Park South West,Mimico NW,The Queensw...",Hotel,Yoga Studio,Fish & Chips Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Falafel Restaurant
9,Northwest,Convenience Store,Hotel,Bank,Pizza Place,Burrito Place,American Restaurant,Theater,Coffee Shop,Clothing Store,Café


In [90]:
etobicoke_grouped.shape

(12, 88)

In [91]:
# set number of clusters
kclusters = 5

etobicoke_grouped_clustering = etobicoke_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 2, 2, 4, 2, 2, 1, 2, 3, 2], dtype=int32)

In [92]:
etobicoke.shape

(12, 5)

In [93]:
etobicoke_merged = etobicoke

# add clustering labels
etobicoke_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
etobicoke_merged = etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

etobicoke_merged.head() # check the last columns!

,Borough,Neighbourhood,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Etobicoke,Islington Avenue,"Etobicoke, M9A",43.744734,-79.239476,0,Ice Cream Shop,Coffee Shop,Convenience Store,Auto Garage,Pizza Place,Sandwich Place,Restaurant,Fast Food Restaurant,Falafel Restaurant,Cuban Restaurant
1,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...","Etobicoke, M9B",43.750072,-79.295849,0,Pizza Place,African Restaurant,Bar,Convenience Store,Coffee Shop,Korean Restaurant,Café,Pharmacy,Burger Joint,Intersection
2,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...","Etobicoke, M9C",43.803762,-79.363452,0,Pharmacy,Park,Residential Building (Apartment / Condo),Sandwich Place,Chinese Restaurant,Pizza Place,Korean Restaurant,Diner,Recreation Center,Ice Cream Shop
3,Etobicoke,Westmount,"Etobicoke, M9P",43.648429,-79.382280,2,Coffee Shop,Café,American Restaurant,Deli / Bodega,Gastropub,Restaurant,Japanese Restaurant,Hotel,Gym,Bar
4,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...","Etobicoke, M9R",43.647927,-79.419750,2,Bar,Café,Coffee Shop,Bakery,Cocktail Bar,Pizza Place,Restaurant,French Restaurant,Asian Restaurant,Vietnamese Restaurant


In [97]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(etobicoke_merged['Latitude'], 
                                  etobicoke_merged['Longitude'], 
                                  etobicoke_merged['Neighbourhood'], 
                                  etobicoke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [95]:
#Examine Clusters
#Cluster 1

etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 0, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Islington Avenue,0,Ice Cream Shop,Coffee Shop,Convenience Store,Auto Garage,Pizza Place,Sandwich Place,Restaurant,Fast Food Restaurant,Falafel Restaurant,Cuban Restaurant
1,"Cloverdale,Islington,Martin Grove,Princess Gar...",0,Pizza Place,African Restaurant,Bar,Convenience Store,Coffee Shop,Korean Restaurant,Café,Pharmacy,Burger Joint,Intersection
2,"Bloordale Gardens,Eringate,Markland Wood,Old B...",0,Pharmacy,Park,Residential Building (Apartment / Condo),Sandwich Place,Chinese Restaurant,Pizza Place,Korean Restaurant,Diner,Recreation Center,Ice Cream Shop


In [96]:
#Cluster 2

etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 1, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Northwest,1,Convenience Store,Hotel,Bank,Pizza Place,Burrito Place,American Restaurant,Theater,Coffee Shop,Clothing Store,Café


In [98]:
#Cluster 3

etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 2, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Westmount,2,Coffee Shop,Café,American Restaurant,Deli / Bodega,Gastropub,Restaurant,Japanese Restaurant,Hotel,Gym,Bar
4,"Kingsview Village,Martin Grove Gardens,Richvie...",2,Bar,Café,Coffee Shop,Bakery,Cocktail Bar,Pizza Place,Restaurant,French Restaurant,Asian Restaurant,Vietnamese Restaurant
6,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",2,Pizza Place,Park,Pub,Gym,Convenience Store,Gas Station,Pharmacy,Skating Rink,Bank,Sandwich Place
7,"Alderwood,Long Branch",2,Pharmacy,Park,Grocery Store,Café,Playground,Skating Rink,Shopping Mall,Bank,Donut Shop,Cuban Restaurant
9,"The Kingsway,Montgomery Road,Old Mill North",2,Convenience Store,Breakfast Spot,Pharmacy,Pizza Place,Fried Chicken Joint,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner
11,"Kingsway Park South West,Mimico NW,The Queensw...",2,Hotel,Yoga Studio,Fish & Chips Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Falafel Restaurant


In [99]:
#Cluster 4

etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 3, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Humber Bay,King's Mill Park,Kingsway Park Sout...",3,Grocery Store,Pizza Place,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Coffee Shop,Pharmacy,Sandwich Place,Hardware Store,Bar


In [100]:
#Cluster 5

etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 4, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"Humber Bay Shores,Mimico South,New Toronto",4,Mexican Restaurant,Falafel Restaurant,Pharmacy,Pizza Place,Café,Bus Stop,Dessert Shop,Liquor Store,Italian Restaurant,Convenience Store
